In [1]:
import matplotlib
#matplotlib.use('Agg')

%load_ext autoreload
%autoreload 2

%matplotlib tk
%autosave 180


import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import os


Autosaving every 180 seconds


/tmp/ipykernel_154562/4197309928.py:12: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
################################################################
############# LOAD DATA AND VISUALIZE AVERAGES #################
################################################################
root_dir = '/home/cat/code/widefield_transformer/data/IA1'

fname_trials = os.path.join(root_dir, 'trials.npy')
d = np.load(fname_trials,
            allow_pickle=True)


#count the number of trials as the first axis of d
n_trials = 0 
for k in range(d.shape[0]):
    n_trials += d[k].shape[0]
print ("# trials: ", n_trials)

# make an array of all the trials to hold data as we loop
all_trials = np.zeros((n_trials, d[0].shape[1], d[0].shape[2]), 'float32')

#
# loop over all the trials and populate the all_trials array

ctr=0
for k in range(d.shape[0]):
    for p in range(d[k].shape[0]):
        all_trials[ctr] = d[k][p]
        ctr+=1

#
print ("all_trials: ", all_trials.shape)

# plot the last trial
plt.figure()
plt.title("Median neural activity for # trials "
          +str(all_trials.shape[0]) + " from mouse IA1")

# make time vector t 
t = np.arange(all_trials.shape[1])/30.-30

for area_id in range(16):
    #plt.plot(all_trials[:,:,area_id].mean(0))
    plt.plot(t, np.median(all_trials[:,:,area_id], 0),
             label='brain area '+str(area_id),
             alpha=.5   )
    

# plot vertical line at t = 0 with max and min y values
plt.plot([0,0], [np.median(all_trials,0).min(), 
                 np.median(all_trials,0).max()], '--',c='black')

# same for horizontal line at y=0
plt.plot([t.min(), t.max()], [0,0], '--',c='black')

#
plt.xlabel('Time (sec)')

#
plt.ylabel("Neurala activity")

#
plt.legend(fontsize=8, ncols=2)
    
plt.show()


# trials:  503
all_trials:  (503, 1800, 16)


### Feeding data into a transformer

We are trying to feed the data into a transfomer model.

The paradigm is roughly to take each trial and feed the first last 15 seconds (from -15 sec to 0 sec) into a transfomer

So the data will be clipped below to only contain time points -15sec .. 0 sec

Then we have 2 options: predict next neural time series time point VS. predicting time to lever pull. Here we only explore the next neural time series time point. There's a separate notebook for option 2.

######################################################################

############### PREDICT NEXT NEURAL TIME SERIES POINT ################

######################################################################
#### Step 1: predict next brain activity

For each trial we take the time points and activity for each brain area and try to predict the next brain activity
- input shape [time_points, brain_area] = [450, 16]
- label: [time_points[1:], brain_area]

We could even start with single brain area
- input shape [time_points]   # for a single area this will look like a time series of length 15 x 30 = 450 time points
                              #    for example:  [30.0, 125.0, -75.3, ... ]
- label: [time_points[1:]   # so here we are predicting the input shifted by 1 [ 125.0, -75.3 ... ]

If this works we can then think about all areas.

Honestly, this is exactly what transformers are developed to do, so we shouldn't have to do too much work to adapt them. We can also smooth or bin the neural data as it's abit noisy.

The major challenges are:

1. Figure out how to convert the brain area activity - which is a float point - to a discretized value
- I think for this I'd suggest taking the range of the neural activity e.g. -200 to + 200 and creating a "vocabulary" in increments of 10. That would give us a 40 token dictionary.

2. It's not clear whether we should feed in snipeets of 15sec as explained above - or we need to randomize the location where we draw the snippets from so the transfomrer doesn't learn some average neural activity. Or maybe this is something that we want!?  Not clear.  In principle we should be able to feed in neural activity from any part of a recording (eg. 30minutes) and get a transformer to perform on this.
- We also have this data, and perhaps we should provide it.
- but for now, we're biasing ourselves by knowing the exact time of the lever pulls




In [ ]:
################################################################
############# LOAD DATA AND VISUALIZE AVERAGES #################
################################################################

In [ ]:
####################################
###### TEST LOCANMF DATA ###########
####################################
data = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb2_30Hz/IA1pm_Feb2_30Hz_locanmf.npz', allow_pickle=True)

trial = data['temporal_trial']
random = data['temporal_random']
print (trial.shape, random.shape)

names = data['names']
print (names)

In [ ]:
######################################################
##### PREDICT SVM DECISION CHOICE SINGLE SESSION #####
######################################################
def predict_ROI_parallel(name,
                         codes):
    
    # 
    svm = SVM.PredictSVMChoice()
    svm.root_dir = data_dir
    svm.random_flag = False  # shuffle data to show baseline

    # window parameters
    svm.window = 30              # prediction window, backwards in time in seconds
    svm.lockout_window = 10      # time for locking out other pulls 
    svm.sliding_window = 30      # number of frames in sliding window
    svm.lockout = False
    svm.gpu_flag = False

    # pca params
    svm.pca_flag = True
    svm.pca_var = 0.95  # this is default for now for nComp = 20

    # svm parameters
    svm.xvalidation = 10  # KFold xvalidation step
    svm.data_split = 0.8  # split 80/20 data for training
    svm.method = 'sigmoid'  # method used for computing SVM

    # run-time parameters
    svm.parallel = True
    svm.n_cores = svm.xvalidation
    svm.min_trials = 10
    svm.overwrite = False


    # session info
    #session_ids = ['Mar1_', 'Mar2_', 'Mar3_', 'Feb29', 'Mar7_']
    svm.session_id = 'all'
    #svm.session_id = 'Feb9_'

    #for name in names:
    svm.animal_id = name

    # 
    for code in codes:
        svm.code = code

        #
        svm.predict_ROI()
        
        
# select animal names
names = ['IA1','IA2','IA3','IJ1','IJ2','AQ2'] # "AR4" and other datasets could work
#names = ['IA1']

# 
codes = ['Retrosplenial', 'barrel', 'limb', 'visual','motor']
codes = ['limb, layer 1 - right', 'limb, layer 1 - left']

#
for name in names:
    predict_ROI_parallel(name,codes)
    
    
    
    

In [ ]:
#################################################################
#################################################################
#################################################################
codes = ['left_paw','right_paw','jaw']

fnames = [
'/media/cat/4TBSSD/yuki/IA1/SVM_Scores/SVM_Scores_IA1pm_Feb5_30Hzjaw_trial_ROItimeCourses_15sec_Xvalid10_Slidewindow30.npz',
'/media/cat/4TBSSD/yuki/IA1/SVM_Scores/SVM_Scores_IA1pm_Feb5_30Hzright_paw_trial_ROItimeCourses_15sec_Xvalid10_Slidewindow30.npz',
'/media/cat/4TBSSD/yuki/IA1/SVM_Scores/SVM_Scores_IA1pm_Feb5_30Hzleft_paw_trial_ROItimeCourses_15sec_Xvalid10_Slidewindow30.npz'
]

for ctr,fname in enumerate(fnames):
    data = np.load(fname)
    acc = data['accuracy']
    print (acc.shape)
    acc = acc[:450]
    t = np.arange(acc.shape[0])/30-14
    plt.plot(t,acc.mean(1),label=codes[ctr])

plt.legend()
plt.show()

In [ ]:
data = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/IA1am_Mar4_30Hz/IA1am_Mar4_30Hz_locanmf.npz',allow_pickle=True)
names = data['names']
#print (names)
for name in names:
    if 'limb, layer 1 - right' in name:
        print (name)
